In [11]:
client_id = "86ac0958ab204b55bf2159768e863ea8"
client_secret = "e99322bd21d2460a94f3401976022220"

In [12]:
import spotipy
import requests
import matplotlib 
from spotipy.oauth2 import SpotifyOAuth
import sqlite3
import time 
import requests 

In [5]:


scope = 'user-read-recently-played'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri = "http://localhost:8888/callback", scope=scope, show_dialog = True ))



In [6]:
#limit = number of tracks it will return, after = is a timestamp in miliseconds(basically songs played '9-5' but we give the time in unix), before = same but before. CANNOT USE BEFORE AND AFTER AT THE SAME TIME
recent = sp.current_user_recently_played(limit=5, after=None, before=None)


In [7]:
print (recent)

{'items': [{'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7vWlb4pM85jCHvV771qZZW'}, 'href': 'https://api.spotify.com/v1/artists/7vWlb4pM85jCHvV771qZZW', 'id': '7vWlb4pM85jCHvV771qZZW', 'name': 'Tony! Toni! Toné!', 'type': 'artist', 'uri': 'spotify:artist:7vWlb4pM85jCHvV771qZZW'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 

In [8]:
# Connect to SQLite database
conn = sqlite3.connect('music_weather.db')
cur = conn.cursor()


In [9]:
# Create tables
cur.execute('''
CREATE TABLE IF NOT EXISTS SpotifyTracks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    track_name TEXT,
    artist TEXT,
    album TEXT,
    played_at INTEGER,
    duration_ms INTEGER
)
''')


In [13]:
# Function to fetch and store recently played tracks
def fetch_spotify_tracks():
    recent_tracks = sp.current_user_recently_played(limit=25)
    for item in recent_tracks['items']:
        track = item['track']
        track_name = track['name']
        artist = track['artists'][0]['name']
        album = track['album']['name']
        played_at = int(time.mktime(time.strptime(item['played_at'][:19], "%Y-%m-%dT%H:%M:%S")))
        duration_ms = track['duration_ms']

        # Insert into database
        cur.execute('''
        INSERT INTO SpotifyTracks (track_name, artist, album, played_at, duration_ms)
        VALUES (?, ?, ?, ?, ?)
        ''', (track_name, artist, album, played_at, duration_ms))

    conn.commit()

# Fetch and store Spotify data
fetch_spotify_tracks()

In [19]:
AP_key = "e2566028033342cebf212759240312"
location = "Ann Arbor"
days = 7
url = f"http://api.weatherapi.com/v1/forecast.json?key={AP_key}&q={location}&days={days}"
response = requests.get(url)
print(response)
data = response.json() 
print(data)

<Response [200]>
{'location': {'name': 'Ann Arbor', 'region': 'Michigan', 'country': 'United States of America', 'lat': 42.2708, 'lon': -83.7264, 'tz_id': 'America/Detroit', 'localtime_epoch': 1733190153, 'localtime': '2024-12-02 20:42'}, 'current': {'last_updated_epoch': 1733189400, 'last_updated': '2024-12-02 20:30', 'temp_c': -1.7, 'temp_f': 28.9, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 7.4, 'wind_kph': 11.9, 'wind_degree': 278, 'wind_dir': 'W', 'pressure_mb': 1027.0, 'pressure_in': 30.32, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 69, 'cloud': 100, 'feelslike_c': -5.8, 'feelslike_f': 21.5, 'windchill_c': -7.6, 'windchill_f': 18.4, 'heatindex_c': -3.5, 'heatindex_f': 25.7, 'dewpoint_c': -5.2, 'dewpoint_f': 22.7, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 14.7, 'gust_kph': 23.7}, 'forecast': {'forecastday': [{'date': '2024-12-02', 'date_epoch': 1733097600, 'day': {'maxtemp